In [1]:
%run supportvectors-common.ipynb



<center><img src="https://d4x5p7s4.rocketcdn.me/wp-content/uploads/2016/03/logo-poster-smaller.png"/> </center>
<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# LangChain Prompts

Good prompts have an overwhelming influence on the quality of the inferences from a large language model. Indeed, over the last few months, there is an increasing realization that the LLMs could potentially start to take over tasks traditionally accomplished through programming -- assuming we can give it well thought out prompts.

<div class="alert-box alert-warning" style="padding-top:30px">
   
<b >Caveat Emptor</b>

> A word of caution is necessary: programming is inherently deterministic, and comprises a clear set of instructions, that yield  -- within certain tolerances -- repeatable results.  </p> On the other hand, large language models are probabilistic machines, and depending on such hyper-parameters as the `temperature`, tend to yield different results on eahc invocation. Here, the value of `temperature` controls the output variations, with `temperature=0` producing close to repeatable results (one reason that this setting is often the default in many situations). On the other hand, higher value, say `temperature=0.9` will tend to produce different results each time.
>
> From our last week's session (our exploration of softmax-temperature), we recall that the temperature is a hyperparameter of the softmax classifier. In other words, when determining the probability of the next word $x_i$ from a vocabulary, the large language models use $$\mathbf{softmax}(x_i/T) = \frac{e^{x_i/T}}{\sum_j e^{x_j/T}}$$ Therefore, the higher the temperature, the more the probability distribution is flattened, and thus a generative model is more likely to sample and pick not necessarily the most likely word, but the next ones in probability. In effect, this leads to a higher variation in the generated text.
<p>
</div>


<hr/>

A `langchain.PromptTemplate` contains two components:

* a template text, with zero or more input-variables embedded in it
* a list of these input variables

Let us now explore a few examples of prompts to develop fluency with them.


### Imports needed for this lab

Let us now import all the components we will need for this lab.

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from IPython.display import Markdown, HTML

Consider a prompt that we may want to give a large language model:

<div class="alert-box alert-info" style="padding-top:30px">
    
> Imagine that there is a story about cats. They like to drink milk and prawl in the neighborhood. Write a short story where some cats show these behaviors.
>
> Give the story a catchy, humorous title. Write it in the format:
>        
>        <h3> [Title] </h3>
>        
>        [Body of the story]
    
</div>
As we learned earlier, we can easily do this with:

In [3]:
prompt = """
        Imagine that there is a story about cats. 
        They like to drink milk and prawl in the neighborhood. 
        Write a short story where some cats show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h3> [Title] </h3>
        
        [Body of the story]
        """

llm = OpenAI(temperature=0.9)
story = llm(prompt)



In [4]:
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h3>The Milk-Sipping, Prowling Adventures of the Cat Gang</h3>

It all began one sunny summer day in the city. The Cat Gang, a group of four feline friends, sat together on a park bench, dreaming of a day full of adventure. 

The Cat Gang decided to start their day off with a light snack. And what better snack than with a little bit of milk? After a quick stop at the local grocery store, the cats were off, sipping their freshly purchased milk as they strutted down the street. 

The Cat Gang, with their bellies full of milk, decided it was time to explore. They weaved their way through backyards and alleyways, sniffing and meowing as they went. Curiosity and excitement filled the air as they were eager to discover new places. 

By the end of their journey, the Cat Gang was exhausted, but they'd had a great day. They had found some cozy spots to nap, and even made a few new furry friends. As the sun set, they made their way home, content with their day of milk-sipping and prowling.</blockquote>

#### Generate variations

Since we set the temperature to `temperature=0.9`, we should expect a different story to emerge the next time we call invoke the model with the same input.


In [5]:
story = llm(prompt)
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h3>Boozy Felines: A Milk-Drinking Adventure</h3>

Once upon a time, there lived a group of cats who loved nothing more than drinking milk and exploring their neighborhood.

Every day, they could be found lounging around the local milk parlor, indulging in luxurious glasses of creamy white goodness. But when it came time to start investigating the neighborhood, they always looked to the one cat who led them all: Captain Meowser.

Captain Meowser was an experienced explorer, having been all around the block and back. With a purr and a meow, he'd lead his feline companions on an adventure, a journey of discovery and excitement. 

As they prowled from street to street, they encountered sights, smells and tastes that left a lasting impression. From the fish market to the local candy shop, no stone was left unturned.

But no matter how far their voyage took them, by nightfall, the cats always had one thing on their minds: milk. Whether it was store-bought or hand-delivered, they couldn't resist a good glass of creamy white liquid, and it was their favorite way to end the evening.</blockquote>

**Observations**

* Note how different the two stories about the cats are. This points to the non-determinism of the results.

* We could have set the temperature to `temperature=0`, and seen results that were a bit more similar, yet not identical. We do that below to verify.


In [6]:
zero_llm = OpenAI(temperature=0.0)

# The first version
display (HTML('<h4>Story of cats: Version 1 </h4>'))
story = llm(prompt)
display(Markdown('<blockquote>' + story + '</blockquote>'))

# The second version
display (HTML('<h4> Story of cats: Version 2 </h4>'))
story = llm(prompt)
display(Markdown('<blockquote>' + story + '</blockquote>'))



<blockquote>
        <h3>The Milk-Loving Cat Posse</h3>

Once upon a time, there lived a group of cats who loved nothing more than relaxing and enjoying a good bowl of milk. Every morning they would all congregate in the alleyway behind the local pet store, each of them chirping and meowing with glee as they waited for their breakfast.

The store owner, an elderly lady by the name of Mrs. Johnson, was happy to oblige. She would let the cats wander inside after each of them had taken their turn lapping up the delicious milk. From there, the cats would often wander through the neighborhood, their bellies full and their spirits high.

They would stop in local gardens, napping in the shade and chasing each other through the tall blades of grass. They darted between the shadows of streetlights and parked cars, often leaping onto car hoods to get a better view of their surroundings.

The cats were happy, and their presence made the neighborhood a little brighter. Everyone who encountered them was charmed, and the cats seemed to know it. 

The cats were known locally as 'The Milk-Loving Cat Posse', and with that,</blockquote>

<blockquote>
<h3>Cats Go Moo-lking Around the Town</h3>

One warm summer night, a group of cats gathered together to go on an adventure. They wanted to explore the neighborhood and find some delicious milk. They put on their matching green collars and set off. 

The cats slipped out of their homes and silently made their way through the streets. They soon spotted a small grocery store with its lights still on, and knew that was their destination. They meowed in triumph, and with one big jump, they were inside the store. 

The cats were amazed at the selection of milk. There was regular milk, almond milk, and even cat milk with extra vitamins. Delighted, the cats started drinking their milk from the cartons. They lapped it up until the cartons were empty. The cats were so full that they just lay down and purred contentedly. 

The cats continued on their adventure, and soon they had explored the entire neighborhood. They found a nearby park and ran around, playing and chasing each other in the grass. As they watched the sun set, they felt fulfilled by the night's adventures. 

The group of cats returned home, bell</blockquote>

## From prompts to templates

Instead of cats, what if we wanted a story about dogs, or about squirrels? A moment of reflection would show that we could have done it with minor variations to our prompt:

**For dogs **

<div class="alert-box alert-info" style="padding-top:30px">   

> Imagine that there is a story about dogs. They like to chew on toys, play fetch and run around the park. Write a short story where some dogs show these behaviors.
>
> Give the story a catchy, humorous title. Write it in the format:
>        
>        <h3> [Title] </h3>
>        
>        [Body of the story]
    
</div>

**For squirrels **

<div class="alert-box alert-info" style="padding-top:30px">   

> Imagine that there is a story about squirrels. They like to eat nuts and fruits, climb up and down trees and run around the glassy meadows. Write a short story where some squirrels show these behaviors.
>
> Give the story a catchy, humorous title. Write it in the format:
>        
>        <h3> [Title] </h3>
>        
>        [Body of the story]
    
</div>

### Capture commonality in templates

Do we see the close similarity between the prompts? Perhaps we would like a lot of stories about animals, and we need to simply give a few descriptive behaviors so the large language models can weave a story around them.

Let us make the above prompts into a template, with two inputs:

* name of the animal
* some characteristic behaviors of the animal


In [7]:
template = """

        Imagine that there is a story about {animal}. 
        They like to {behaviors}. 
        Write a short story where some {animal} show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h4> [Title] </h4>
        
        [Body of the story]

        """


Then we can generate our prompt for a story about dogs with:

In [8]:
prompt = PromptTemplate(
            input_variables = ['animal', 'behaviors'],
            template        = template
)

Then, for a story about dogs, we can generate the prompt text as follows:

In [9]:
dogs = 'dogs'
dog_behaviors = 'chew on toys, play fetch and run around the park'

for_dogs = prompt.format(animal = dogs, behaviors = dog_behaviors)
print(for_dogs)



        Imagine that there is a story about dogs. 
        They like to chew on toys, play fetch and run around the park. 
        Write a short story where some dogs show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h4> [Title] </h4>
        
        [Body of the story]

        


Likewise, for squirrels, we could get the prompt:

In [10]:
squirrels = 'squirrels'
squirrel_behaviors = """eat nuts and fruits, 
        climb up and down trees and run around the glassy meadows"""

for_squirrels = prompt.format(animal = squirrels, behaviors = squirrel_behaviors)
print (for_squirrels)



        Imagine that there is a story about squirrels. 
        They like to eat nuts and fruits, 
        climb up and down trees and run around the glassy meadows. 
        Write a short story where some squirrels show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h4> [Title] </h4>
        
        [Body of the story]

        


#### A simple LangChain 

Let us concatenate the `promptTemplate` and the `llm` into a single chain, so we can invoke it with needing to invoke each component, and feeding its output to the next component.

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

And now we can invoke it to, say, get a story about dogs:

In [12]:
story = chain.run (animal=dogs, behaviors=dog_behaviors)
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h4>The Adventures of Fetch and Chew</h4>

Once upon a time, there were two dogs named Fetch and Chew. Both were full of energy, and they spent their days playing together in the park.

Fetch loved nothing more than chasing a stick and bringing it back to his master. He could run around the park for hours, racing from one side to the other, and always coming back with a smile on his face. 

Chew, on the other hand, loved nothing more than playing with his favorite toys. He would carry them around in his mouth, shaking his head and growling whenever someone tried to take them away. No matter how many times he chomped down on them, they were never too worse for wear. 

On this particular day, the two dogs were happily playing together in the park. Fetch would race around, looking for the perfect stick, while Chew chased around his favorite toys. They were both having a great time, and the sun was shining brightly. 

The two dogs were so engaged in their play that they didn't notice the time slipping away. Soon, it was time to go home, but Fetch and Che</blockquote>

Likewise, a story about squirrels with:

In [13]:
story = chain.run (animal=squirrels, behaviors=squirrel_behaviors)
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h4> Nut-tastic Adventures of the Squirrels </h4>

It was a beautiful day and the squirrels were out in full force. There were little furry bodies darting around in the trees and racing across the meadows.

The most daring squirrels were out in search of food. They were up in the trees, collecting nuts and fruits and eating them with gusto. Some of them were so daring they would climb to dizzying heights and then jump down to the ground, barely missing the nearby bushes.

Others were up and down the trees like a yo-yo, scurrying up and down the trunk and swaying from branch to branch. The trees seemed to shudder in delight as the squirrels navigated their barky surfaces.

Meanwhile, some of the squirrels were running across the glassy meadows, kicking up clouds of dust and making their tails dance. The wind seemed to whistle with joy as the squirrels raced each other and chased invisible foes across the meadow.

All in all, it was a nut-tastic day for the squirrels, with plenty of daring adventures, enjoyable goodies, and lots of fun.</blockquote>

**Note**
While a `LLMChain` will take a prompt-template as an argument, it is not necessary to provide a user input. For example, the following prompt template does not take any user input, and still forms a part of the chain, that we can invoke.

In [14]:
simple_prompt = PromptTemplate(
    template="A witty quote on everyday life in contemporary times.",
    input_variables=[] # we cannot omit this argument, even if empty
)
print (simple_prompt.format())

quote = llm(simple_prompt.format())
display(Markdown('<blockquote>' + quote + '</blockquote>'))

A witty quote on everyday life in contemporary times.


<blockquote>

"Life moves pretty fast. If you don't stop and look around once in a while, you might miss it." - Ferris Bueller</blockquote>

#### Chain with a simple prompt

We can run the chain with a simple prompt that needs no user input as follows:

In [15]:
chain = LLMChain(llm=llm, prompt=simple_prompt)
witty_quote = chain.run (input="")
print(witty_quote)



“Life is 10% what happens to us and 90% how we react to it.” — Charles R. Swindoll


## Few shot examples

It is often the case that we want to give the large language models a few exemplars, so that it can infer the relationships. Let us illustrate it with an example.

In [16]:
template = """
Animal: {animal}
Likes to: {likes_to}
"""

examples = [
    {'animal': 'dog', 'likes_to': 'play fetch!'},
    {'animal': 'cat', 'likes_to': 'prawl around the neighborhood'},
    {'animal': 'squirrel', 'likes_to': 'eat nuts and climb trees'}
]


Let us now create an example prompt from this:

In [17]:
example_prompt = PromptTemplate(
    input_variables=['animal', 'likes_to'],
    template = template)


Now, let's create the `FewShotPromptTemplate` with these two.

In [18]:
few_shot_prompt       = FewShotPromptTemplate(
    examples          = examples,
    example_prompt    = example_prompt,
    prefix            = 'Describe the behavior of every animal input',
    suffix            = 'Animal: {input}\nLikes to:',
    input_variables   = ['input'],
    example_separator = '\n\n'
)

We can generate the prompt text from the above:

In [19]:
text = few_shot_prompt.format(input='horse')
print(text)

Describe the behavior of every animal input


Animal: dog
Likes to: play fetch!



Animal: cat
Likes to: prawl around the neighborhood



Animal: squirrel
Likes to: eat nuts and climb trees


Animal: horse
Likes to:


In [20]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)
chain.run('horse')

' eat hay and run around in pastures'

**Summary**

We have learned so far that a `prompt` in `langchain` is the input that goes into a large language model. As we will learn later, there are prompts for other things too, such as chat models, and so forth.

In general, a `prompt` is not necessarily a pre-determined string, but can comprise:

* a template (`PromptTemplate`), with  user input variables embedded
* user inputs for each these variables
* and also some examples


## Partial Prompt Template

Sometimes, we can construct a simpler prompt template by taking one that needs multiple user inputs, but hardwiring some (but not all) of these. Here is an example:

In [21]:
template = """
    Name three historically significant events in {country} around the year {year}
"""

prompt = PromptTemplate (
            input_variables = ['country', 'year'],
            template        = template,
)

print(prompt.format(country='England', year='1800'))


    Name three historically significant events in England around the year 1800



In [22]:
chain = LLMChain(
    llm=llm,
    prompt=prompt
)

history = chain.run(country='England', year='1800')
print(history)


1. The Battle of Trafalgar (October 21, 1805): British naval victory under Admiral Lord Nelson against the combined forces of the French and Spanish fleets during the Napoleonic Wars.

2. The Industrial Revolution (1760-1830): This period of time saw the introduction of new technologies and an increase in the production of goods, the large-scale investment in new forms of manufacturing and the growth of the British Empire.

3. The Great Reform Act of 1832: This act represented a major shift in the British political landscape, introducing large-scale changes to the electorate and House of Commons. It was a major step in the evolution of the modern parliamentary system in the United Kingdom.


Let us now suppose that we are particularly interested in the French history.
Then, we can create a simpler, partial prompt out of the above:

In [23]:
partial_prompt = prompt.partial(country='France')
print(partial_prompt.format(year='1789'))


    Name three historically significant events in France around the year 1789



And we can feed it into a language chain as follows:

In [24]:
chain = LLMChain(
            llm     = llm,
            prompt  = partial_prompt
)

events = chain.run(year='1789')
print(events)


1. Storming of the Bastille on July 14, 1789
2. Estates-General Becomes the National Assembly on June 17, 1789
3. Declaration of the Rights of Man and of the Citizen on August 26, 1789


## What we have learned?

In this lab, we learned about:

* what is a `langchain.PromptTemplate`, and what purpose it serves
* many illustrations of constructing the prompt templates
* using prompt templates with few shot examples
* judicious choice of prompt template
